In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import glob
import os
import numpy as np

# Device Setup
device = torch.device("cuda:4")
print(f"Using device: {device}")

# Define the Autoencoder Encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
    
    def forward(self, x):
        return self.encoder(x)

# Define the Transfer Learning Model
class TransferModel(nn.Module):
    def __init__(self, encoder, classifier):
        super(TransferModel, self).__init__()
        self.encoder = encoder
        self.classifier = classifier
        
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten before classifier
        x = self.classifier(x)
        return x

# Load Pretrained Encoder
encoder = Encoder()
encoder.load_state_dict(torch.load("autoencoder.pth", map_location=device), strict=False)
encoder.to(device)
encoder.eval()

# Freeze Encoder Layers
for param in encoder.parameters():
    param.requires_grad = False

# Define the Classifier from the CNN Model
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(64 * 28 * 28, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 8)
        )
    
    def forward(self, x):
        return self.classifier(x)

# Create Model
classifier = Classifier().to(device)
model = TransferModel(encoder, classifier).to(device)

# Define Training Parameters
criterion = nn.MSELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4, weight_decay=1e-4)

# Training Function
def train_model(model, train_loader, val_loader, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, targets in train_loader:
            images, targets = images.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, targets in val_loader:
                images, targets = images.to(device), targets.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, targets).item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss/len(train_loader):.6f} | Val Loss: {val_loss/len(val_loader):.6f}")
    
    torch.save(model.state_dict(), 'transfer_model.pth')
    print("Model saved as transfer_model.pth")

# Data Loader Function
def get_data_loaders(root_folder, batch_size=40, num_workers=12):
    file_paths = glob.glob(os.path.join(root_folder, '*.jpg'))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale(1),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    class Dataset(torch.utils.data.Dataset):
        def __init__(self, file_paths, transform):
            self.file_paths = file_paths
            self.transform = transform
        
        def __len__(self):
            return len(self.file_paths)
        
        def __getitem__(self, idx):
            image = Image.open(self.file_paths[idx])
            image = self.transform(image)
            return image, torch.zeros(8)  # Placeholder target
    
    dataset = Dataset(file_paths, transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader

def evaluate_model(model, val_loader):
    print("Evaluating model on validation set...")
    model.eval()
    with torch.no_grad():
        for images, actual_params in val_loader:
            images, actual_params = images.to(device), actual_params.to(device)
            predictions = model(images)
            break  # Display only the first batch

    # Denormalize predictions and actual values
    predictions = predictions.cpu().numpy() * 5 # Scale back to [-5, 5]
    actual_params = actual_params.cpu().numpy() * 5  # Scale back to [-5, 5]

    for i in range(min(5, predictions.shape[0])):
        print(f"\nSample {i+1}:")
        print(f"Predicted: {predictions[i]}")
        print(f"Actual:    {actual_params[i]}")
        print(f"Abs Error: {np.abs(predictions[i] - actual_params[i])}")

# Main Execution
if __name__ == '__main__':
    train_loader, val_loader = get_data_loaders('training', batch_size=40, num_workers=12)
    train_model(model, train_loader, val_loader, num_epochs=20)
    evaluate_model(model, val_loader)


Using device: cuda:4
Epoch [1/20] | Train Loss: 0.090618 | Val Loss: 0.008445
Epoch [2/20] | Train Loss: 0.056166 | Val Loss: 0.004962
Epoch [3/20] | Train Loss: 0.045306 | Val Loss: 0.003025
Epoch [4/20] | Train Loss: 0.038787 | Val Loss: 0.002085
Epoch [5/20] | Train Loss: 0.032296 | Val Loss: 0.001777
Epoch [6/20] | Train Loss: 0.028893 | Val Loss: 0.001787
Epoch [7/20] | Train Loss: 0.025621 | Val Loss: 0.001291
Epoch [8/20] | Train Loss: 0.022975 | Val Loss: 0.001152
Epoch [9/20] | Train Loss: 0.019948 | Val Loss: 0.000931
Epoch [10/20] | Train Loss: 0.017492 | Val Loss: 0.000841
Epoch [11/20] | Train Loss: 0.016149 | Val Loss: 0.000693
Epoch [12/20] | Train Loss: 0.013857 | Val Loss: 0.000616
Epoch [13/20] | Train Loss: 0.012671 | Val Loss: 0.000644
Epoch [14/20] | Train Loss: 0.011375 | Val Loss: 0.000542
Epoch [15/20] | Train Loss: 0.010749 | Val Loss: 0.000519
Epoch [16/20] | Train Loss: 0.009380 | Val Loss: 0.000538
Epoch [17/20] | Train Loss: 0.008294 | Val Loss: 0.000475
Ep

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import glob
import os
import numpy as np
import re

# Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the Autoencoder Encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
    
    def forward(self, x):
        return self.encoder(x)

# Define the Transfer Learning Model
class TransferModel(nn.Module):
    def __init__(self, encoder, classifier):
        super(TransferModel, self).__init__()
        self.encoder = encoder
        self.classifier = classifier
        
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten before classifier
        x = self.classifier(x)
        return x

# Load Pretrained Encoder
encoder = Encoder()
encoder.load_state_dict(torch.load("autoencoder.pth", map_location=device), strict=False)
encoder.to(device)
encoder.eval()

# Freeze Encoder Layers
for param in encoder.parameters():
    param.requires_grad = False

# Define the Classifier from the CNN Model
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(64 * 28 * 28, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 8)
        )
    
    def forward(self, x):
        return self.classifier(x)

# Create Model
classifier = Classifier().to(device)
model = TransferModel(encoder, classifier).to(device)

# Define Training Parameters
criterion = nn.MSELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4, weight_decay=1e-4)

# Training Function
def train_model(model, train_loader, val_loader, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, targets in train_loader:
            images, targets = images.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, targets in val_loader:
                images, targets = images.to(device), targets.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, targets).item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss/len(train_loader):.6f} | Val Loss: {val_loss/len(val_loader):.6f}")
    
    torch.save(model.state_dict(), 'transfer_model.pth')
    print("Model saved as transfer_model.pth")

# Evaluation Function
def evaluate_model(model, val_loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(val_loader)
    print(f"Evaluation Loss: {avg_loss:.6f}")
    
    # Display sample predictions
    sample_images, sample_targets = next(iter(val_loader))
    sample_images, sample_targets = sample_images.to(device), sample_targets.to(device)
    sample_outputs = model(sample_images).detach()
    
    for i in range(min(5, sample_outputs.size(0))):
        predicted = sample_outputs[i].cpu().numpy()
        actual = sample_targets[i].cpu().numpy()
        abs_error = abs(predicted - actual)
        print(f"\nSample {i+1}:")
        print(f"Predicted: {predicted}")
        print(f"Actual:    {actual}")
        print(f"Abs Error: {abs_error}")

# Data Loader Function
def get_data_loaders(root_folder, batch_size=40, num_workers=12):
    file_paths = glob.glob(os.path.join(root_folder, '*.jpg'))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale(1),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    class Dataset(torch.utils.data.Dataset):
        def __init__(self, file_paths, transform):
            self.file_paths = file_paths
            self.transform = transform
        
        def __len__(self):
            return len(self.file_paths)
        
        def __getitem__(self, idx):
            image = Image.open(self.file_paths[idx])
            image = self.transform(image)
            
            # Extract coefficients from filename
            name = os.path.splitext(os.path.basename(self.file_paths[idx]))[0]
            name = name.replace('n', '-').replace('p', '.')
            parts = re.findall(r'[-+]?[0-9]*\.?[0-9]+', name)
            params = np.array([float(p) for p in parts[:8]])  # Extract first 8 coefficients
            params = torch.FloatTensor(params)
            
            return image, params
    
    dataset = Dataset(file_paths, transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader

# Main Execution
if __name__ == '__main__':
    train_loader, val_loader = get_data_loaders('training2', batch_size=40, num_workers=12)
    train_model(model, train_loader, val_loader, num_epochs=20)
    evaluate_model(model, val_loader)


Using device: cuda
Epoch [1/20] | Train Loss: 8.392342 | Val Loss: 8.313062
Epoch [2/20] | Train Loss: 8.354016 | Val Loss: 8.313567
Epoch [3/20] | Train Loss: 8.347391 | Val Loss: 8.314226
Epoch [4/20] | Train Loss: 8.342511 | Val Loss: 8.312906
Epoch [5/20] | Train Loss: 8.339682 | Val Loss: 8.311634
Epoch [6/20] | Train Loss: 8.336941 | Val Loss: 8.311463
Epoch [7/20] | Train Loss: 8.336533 | Val Loss: 8.310827
Epoch [8/20] | Train Loss: 8.335437 | Val Loss: 8.311628
Epoch [9/20] | Train Loss: 8.334539 | Val Loss: 8.310966
Epoch [10/20] | Train Loss: 8.334490 | Val Loss: 8.309077
Epoch [11/20] | Train Loss: 8.333390 | Val Loss: 8.310103
Epoch [12/20] | Train Loss: 8.333456 | Val Loss: 8.310214
Epoch [13/20] | Train Loss: 8.332027 | Val Loss: 8.309866
Epoch [14/20] | Train Loss: 8.332919 | Val Loss: 8.309618
Epoch [15/20] | Train Loss: 8.332160 | Val Loss: 8.309018
Epoch [16/20] | Train Loss: 8.333633 | Val Loss: 8.308921
Epoch [17/20] | Train Loss: 8.331893 | Val Loss: 8.310053
Epoc